<a href="https://colab.research.google.com/github/Yashikatahaliyani-03/Basics_on_python/blob/main/Module_2_Lab_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "view-in-github",
        "colab_type": "text"
      },
      "source": [
        "<a href=\"https://colab.research.google.com/github/akkkiii08/FMML_Projects-and-labs/blob/main/Module2_Lab4_ML_Akshay.ipynb\" target=\"_parent\"><img src=\"https://colab.research.google.com/assets/colab-badge.svg\" alt=\"Open In Colab\"/></a>"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "hZ3ju1NR18I7"
      },
      "source": [
        "Module 2: Appreciating, Interpreting and Visualizing Data\n",
        "\n",
        "Lab 4: t-Distributed Stochastic Neighbor Embedding (t-SNE)"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "w7sDk_k62LgK"
      },
      "outputs": [],
      "source": [
        "# numpy.\n",
        "import numpy as np\n",
        "from numpy import linalg\n",
        "from numpy.linalg import norm\n",
        "from scipy.spatial.distance import squareform, pdist\n",
        "\n",
        "# sklearn.\n",
        "import sklearn\n",
        "from sklearn.manifold import TSNE\n",
        "from sklearn.datasets import load_digits\n",
        "from sklearn.preprocessing import scale\n",
        "\n",
        "# Random state.\n",
        "RS = 20150101\n",
        "\n",
        "# matplotlib.\n",
        "import matplotlib.pyplot as plt\n",
        "import matplotlib.patheffects as PathEffects\n",
        "import matplotlib\n",
        "%matplotlib inline\n",
        "\n",
        "# seaborn.\n",
        "import seaborn as sns\n",
        "sns.set_style('darkgrid')\n",
        "sns.set_palette('muted')\n",
        "sns.set_context(\"notebook\", font_scale=2.5,\n",
        "                rc={\"lines.linewidth\": 3.5})"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "MRNcpOsd2vnB"
      },
      "source": [
        "1797 images each of size 8 x 8 loaded using load_digits()"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "uueQffp_2xam",
        "outputId": "c50ff550-7ec3-4448-ace9-3eb0e5597e82"
      },
      "outputs": [
        {
          "output_type": "execute_result",
          "data": {
            "text/plain": [
              "(1797, 64)"
            ]
          },
          "metadata": {},
          "execution_count": 2
        }
      ],
      "source": [
        "digits = load_digits()\n",
        "digits.data.shape"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 417
        },
        "id": "uzxSfDCL3Mch",
        "outputId": "e53d4f38-3793-46e0-9eda-ce8cb95b6e1c"
      },
      "outputs": [
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 600x500 with 18 Axes>"
            ],
            "image/png": "...\n"
          },
          "metadata": {}
        }
      ],
      "source": [
        "nrows, ncols = 3, 6\n",
        "plt.figure(figsize=(6,5))\n",
        "plt.gray()\n",
        "for i in range(ncols * nrows):\n",
        "    ax = plt.subplot(nrows, ncols, i + 1)\n",
        "    ax.matshow(digits.images[i,...])\n",
        "    plt.xticks([]); plt.yticks([])\n",
        "    plt.title(digits.target[i])"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "NpF9G-tI3ksS"
      },
      "outputs": [],
      "source": [
        "# We first reorder the data points according to the handwritten numbers.\n",
        "X = np.vstack([digits.data[digits.target==i]\n",
        "               for i in range(10)])\n",
        "y = np.hstack([digits.target[digits.target==i]\n",
        "               for i in range(10)])"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "A9OcmQAu31lA"
      },
      "source": [
        "Now using TSNE to fit the dataset with the default values.\n",
        "\n",
        "1.n_components : 2\n",
        "\n",
        "2.perplexity : 30\n",
        "\n",
        "3.n_iter : 1000\n",
        "\n",
        "4.method : ‘barnes_hut’\n"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "id": "RRtiiltb4FRu"
      },
      "outputs": [],
      "source": [
        "digits_proj = TSNE(init=\"pca\", random_state=RS).fit_transform(X)"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 707
        },
        "id": "W-5lfa8i4WeO",
        "outputId": "a196b1a8-8e8d-458a-dad3-e0dccf7afd39"
      },
      "outputs": [
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 800x800 with 1 Axes>"
            ],
            "image/png": "...\n"
          },
          "metadata": {}
        },
        {
          "output_type": "execute_result",
          "data": {
            "text/plain": [
              "(<Figure size 800x800 with 1 Axes>,\n",
              " <Axes: >,\n",
              " <matplotlib.collections.PathCollection at 0x7988be652ef0>)"
            ]
          },
          "metadata": {},
          "execution_count": 10
        }
      ],
      "source": [
        "def scatter(x, colors):\n",
        "    # We choose a color palette with seaborn.\n",
        "    palette = np.array(sns.color_palette(\"hls\", 10))\n",
        "\n",
        "    # We create a scatter plot.\n",
        "    f = plt.figure(figsize=(8, 8))\n",
        "    ax = plt.subplot(aspect='equal')\n",
        "    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40,\n",
        "                    c=palette[colors.astype(int)])\n",
        "    plt.xlim(-75, 75)\n",
        "    plt.ylim(-75, 75)\n",
        "    ax.axis('off')\n",
        "    ax.axis('tight')\n",
        "\n",
        "    # We add the labels for each digit.\n",
        "    txts = []\n",
        "    for i in range(10):\n",
        "        # Position of each label.\n",
        "        xtext, ytext = np.median(x[colors == i, :], axis=0)\n",
        "        txt = ax.text(xtext, ytext, str(i), fontsize=24)\n",
        "        txt.set_path_effects([\n",
        "            PathEffects.Stroke(linewidth=5, foreground=\"w\"),\n",
        "            PathEffects.Normal()])\n",
        "        txts.append(txt)\n",
        "    plt.show()\n",
        "\n",
        "    return f, ax, sc\n",
        "\n",
        "scatter(digits_proj, y)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "HZqcLiNT42cU"
      },
      "source": [
        "Tweaking some of the hyperparameters to better understand their role\n",
        "\n",
        "Changing the PERPLEXITY values"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 707
        },
        "id": "I3jtw69c4522",
        "outputId": "0d80f311-daee-42c5-f4a5-9e760b3a35e1"
      },
      "outputs": [
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 800x800 with 1 Axes>"
            ],
            "image/png": "...\n"
          },
          "metadata": {}
        },
        {
          "output_type": "execute_result",
          "data": {
            "text/plain": [
              "(<Figure size 800x800 with 1 Axes>,\n",
              " <Axes: >,\n",
              " <matplotlib.collections.PathCollection at 0x7988bc5c83d0>)"
            ]
          },
          "metadata": {},
          "execution_count": 11
        }
      ],
      "source": [
        "digits_proj = TSNE(init=\"pca\", random_state=RS, perplexity=7).fit_transform(X)\n",
        "\n",
        "scatter(digits_proj, y)"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 707
        },
        "id": "SgD6QFqb5Q6c",
        "outputId": "916d07c0-2a76-471a-cee2-5ed3a4061a46"
      },
      "outputs": [
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 800x800 with 1 Axes>"
            ],
            "image/png": "...\n"
          },
          "metadata": {}
        },
        {
          "output_type": "execute_result",
          "data": {
            "text/plain": [
              "(<Figure size 800x800 with 1 Axes>,\n",
              " <Axes: >,\n",
              " <matplotlib.collections.PathCollection at 0x7988f75feb90>)"
            ]
          },
          "metadata": {},
          "execution_count": 12
        }
      ],
      "source": [
        "digits_proj = TSNE(init=\"pca\", random_state=RS, perplexity=400).fit_transform(X)\n",
        "\n",
        "scatter(digits_proj, y)"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 707
        },
        "id": "PyOi3F4D88tj",
        "outputId": "a0be09ac-eb03-483e-c237-3d24e6aafd0a"
      },
      "outputs": [
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 800x800 with 1 Axes>"
            ],
            "image/png": "...\n"
          },
          "metadata": {}
        },
        {
          "output_type": "execute_result",
          "data": {
            "text/plain": [
              "(<Figure size 800x800 with 1 Axes>,\n",
              " <Axes: >,\n",
              " <matplotlib.collections.PathCollection at 0x7988be680490>)"
            ]
          },
          "metadata": {},
          "execution_count": 13
        }
      ],
      "source": [
        "digits_proj = TSNE(init=\"pca\", random_state=RS, n_iter=450).fit_transform(X)\n",
        "\n",
        "scatter(digits_proj, y)"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 707
        },
        "id": "UU2YCQH_9ti4",
        "outputId": "5239d2e4-fef9-4bd1-cb29-6322201c0cdd"
      },
      "outputs": [
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 800x800 with 1 Axes>"
            ],
            "image/png": "...\n"
          },
          "metadata": {}
        },
        {
          "output_type": "execute_result",
          "data": {
            "text/plain": [
              "(<Figure size 800x800 with 1 Axes>,\n",
              " <Axes: >,\n",
              " <matplotlib.collections.PathCollection at 0x7988bbd3b760>)"
            ]
          },
          "metadata": {},
          "execution_count": 14
        }
      ],
      "source": [
        "digits_proj = TSNE(init=\"pca\", random_state=RS, n_iter=8000).fit_transform(X)\n",
        "\n",
        "scatter(digits_proj, y)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "Q8o7tupA-k4j"
      },
      "source": [
        "Changing the method to EXACT\n",
        "\n",
        "The barnes-hut method takes O(NlogN) time, whereas the exact method takes O(N^2) time. Notice the increase in the execution time while running the cell below."
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 707
        },
        "id": "chbBmlcf-tWE",
        "outputId": "a9c27a99-5d0a-4751-bea6-d1db93fa3c14"
      },
      "outputs": [
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 800x800 with 1 Axes>"
            ],
            "image/png": "...\n"
          },
          "metadata": {}
        },
        {
          "output_type": "execute_result",
          "data": {
            "text/plain": [
              "(<Figure size 800x800 with 1 Axes>,\n",
              " <Axes: >,\n",
              " <matplotlib.collections.PathCollection at 0x7988bbdd5120>)"
            ]
          },
          "metadata": {},
          "execution_count": 15
        }
      ],
      "source": [
        "digits_proj = TSNE(init=\"pca\", random_state=RS, method='exact').fit_transform(X)\n",
        "\n",
        "scatter(digits_proj, y)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "SwKQORz_AMv5"
      },
      "source": [
        "Exercise"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 707
        },
        "id": "JNFWFAHy_pkK",
        "outputId": "9a1428c0-1980-4247-ff84-601e19e2829d"
      },
      "outputs": [
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "<Figure size 800x800 with 1 Axes>"
            ],
            "image/png": "...\n"
          },
          "metadata": {}
        },
        {
          "output_type": "execute_result",
          "data": {
            "text/plain": [
              "(<Figure size 800x800 with 1 Axes>,\n",
              " <Axes: >,\n",
              " <matplotlib.collections.PathCollection at 0x7988beb536a0>)"
            ]
          },
          "metadata": {},
          "execution_count": 16
        }
      ],
      "source": [
        "###############################################################\n",
        "###### MODIFY THE VALUES FOR THE HYPERPARAMETERS BELOW ########\n",
        "\n",
        "perplexity_value = 1\n",
        "number_iterations = 4000\n",
        "method = \"barnes_hut\"\n",
        "\n",
        "###############################################################\n",
        "\n",
        "digits_proj = TSNE(init=\"pca\", random_state=RS, n_iter=number_iterations, perplexity=perplexity_value, method=method).fit_transform(X)\n",
        "scatter(digits_proj, y)"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "yAo3_EzNBlf_"
      },
      "source": [
        "QUESTION:\n",
        "\n",
        "Now that you understand t-SNE a bit better, can you point out some differences between PCA and t-SNE. What are the advantages/disadvantages of one over the other?\n",
        "\n",
        "Pointers:\n",
        "\n",
        "1.Which of the two algorithms is linear and which one is non-linear?\n",
        "\n",
        "2.How does the non-linearity in one of these two algorithms help in capturing certain data sets?\n",
        "\n",
        "3.PCA is known to keep points which were further apart in the higher dimension, far apart in the lower dimension as well. Does t-SNE do the same? Or does it try to preserve local neighbourhood?\n",
        "\n",
        "4.Can you comment on which one of the two is computationally more expensive?\n"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "ugiB0ZkbCEGt"
      },
      "source": [
        "ANSWER :\n"
      ]
    }
  ],
  "metadata": {
    "colab": {
      "provenance": [],
      "include_colab_link": true
    },
    "kernelspec": {
      "display_name": "Python 3",
      "name": "python3"
    },
    "language_info": {
      "name": "python"
    }
  },
  "nbformat": 4,
  "nbformat_minor": 0
}